# 数据预处理

数据清洗、集成、转换、规约等一系列处理

在数据挖掘中，海量的原始数据中存在着大量不完整（有缺失值）、不一致、有异常的数据，严重影响到数据挖掘建模的执行效率，甚至可能导致挖掘结果的偏差，所以进行数据清洗尤为重要。

- 提高数据的质量
- 让数据更好地适应特定的挖掘技术或者工具
- 占据整个数据挖掘时间的约60%

## 数据清洗

### 缺失值处理

- 删除记录
- 将缺失值视为一类
- 均值/中位数/众数插补
- 使用特定值插补
- 最近邻插补
- 回归插补
- 插值函数插补
- 拉格朗日插值法
- 牛顿插值法

In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import lagrange

In [2]:
inputfile = './data/data_4/catering_sale.xls'
outputfile = './data/data_4/sales/xls'

data = pd.read_excel(inputfile)
data.loc[(data['销量'] < 400) | (data['销量'] > 5000), '销量'] = np.nan

def polyinterp_column(s, n, k=5):
    y = s[list(range(n-k, n)) + list(range(n+1, n+k+1))]
    y = y[y.notnull()]
    return lagrange(y.index, list(y))(n)

for i in data.columns[1:]:
    for j in data.index:
        if data[i].isnull()[j] == True:
            data.loc[j, i] = polyinterp_column(data[i], j)
data.isnull().sum()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\series.py:842: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


日期    0
销量    0
dtype: int64

### 异常值处理

- 删除含有异常值的记录
- 视为缺失值，利用处理缺失值的方法处理
- 平均值修正：用前后两个数值的平均值替代异常值
- 不处理，直接带着异常值进行挖掘

**在很多情况下，需要先对异常值产生的原因进行分析，才能选择适当的异常值处理方法。如果是正常记录下的数据，那么往往不需要处理。**

## 数据集成

数据挖掘需要的数据往往分布在不同的数据源中，数据集成就是将多个数据源合并存放在一个一直的数据存储中的过程。

### 实体识别

统一不同数据源的矛盾之处：
- 同名异义
- 异名同义
- 单位不统一

### 冗余属性识别

- 同一属性多次出现
- 同一属性命名不一致导致重复

有时可以用相关分析来发现冗余属性，**相关系数在一定程度上度量了一个属性在多大程度上蕴含了另一个属性的信息**。

## 数据变换

对数据进行规范化处理，将数据转换成“适当”的形式，适合挖掘任务及算法的需要。

### 简单函数变换

常用来将不具有正太分布的数据变换成具有正态分布的数据
- 平方变换
- 开方变换
- 对数变换
- 差分变换

### 规范化

为了消除指标之间的量纲和取值范围差异的影响， 将数据按照比例进行缩放，使之落入一个特定的区域，便于进行综合分析。**数据规范化对于基于距离的算法尤为重要**。

- 最大-最小规范化
- 零-均值规范化
- 小数定标规范化

In [3]:
datafile = './data/data_4/normalization_data.xls'
data = pd.read_excel(datafile, header=None)
data

,0,1,2,3
0,78,521,602,2863
1,144,-600,-521,2245
2,95,-457,468,-1283
3,69,596,695,1054
4,190,527,691,2051
5,101,403,470,2487
6,146,413,435,2571


In [4]:
(data - data.min()) / (data.max() - data.min())

,0,1,2,3
0,0.074380,0.937291,0.923520,1.000000
1,0.619835,0.000000,0.000000,0.850941
2,0.214876,0.119565,0.813322,0.000000
3,0.000000,1.000000,1.000000,0.563676
4,1.000000,0.942308,0.996711,0.804149
5,0.264463,0.838629,0.814967,0.909310
6,0.636364,0.846990,0.786184,0.929571


In [5]:
(data - data.mean()) / data.std()

,0,1,2,3
0,-0.905383,0.635863,0.464531,0.798149
1,0.604678,-1.587675,-2.193167,0.369390
2,-0.516428,-1.304030,0.147406,-2.078279
3,-1.111301,0.784628,0.684625,-0.456906
4,1.657146,0.647765,0.675159,0.234796
5,-0.379150,0.401807,0.152139,0.537286
6,0.650438,0.421642,0.069308,0.595564


In [6]:
data / (10 ** np.ceil(np.log10(data.abs().max())))

,0,1,2,3
0,0.078,0.521,0.602,0.2863
1,0.144,-0.600,-0.521,0.2245
2,0.095,-0.457,0.468,-0.1283
3,0.069,0.596,0.695,0.1054
4,0.190,0.527,0.691,0.2051
5,0.101,0.403,0.470,0.2487
6,0.146,0.413,0.435,0.2571


### 连续属性离散化

一些数据挖掘算法（如ID3算法，Apriori算法等）要求数据是分类属性形式，因此需要将连续属性转换成分类属性，即连续属性离散化。

离散化的过程包括确定分类数以及将连续属性映射到这些分类值。

- 等宽法：将属性的值域分为具有相同宽度的区间，容易受到离群点的影响
- 等频法：将相同数量的记录放进每个区间，不易受到离群点的影响，但是可能将同一值的点分到不同区间
- 基于聚类分析的方法：先进行聚类，然后将一个簇的属性值做同一标记，需要指定簇的个数

In [7]:
datafile = './data/data_4/discretization_data.xls'
data = pd.read_excel(datafile)
data = data['肝气郁结证型系数'].copy()
k = 4

In [8]:
pd.cut(data, k, labels=False).head()

0    0
1    3
2    0
3    2
4    1
Name: 肝气郁结证型系数, dtype: int64

In [9]:
pd.qcut(data, k, labels=False).head()

0    0
1    3
2    0
3    3
4    2
Name: 肝气郁结证型系数, dtype: int64

In [10]:
from sklearn.cluster import KMeans

kmodel = KMeans(n_clusters=k).fit(data.values.reshape(-1, 1))

clusters = pd.Series(kmodel.cluster_centers_.flatten()).sort_values()
for i in range(len(clusters)):
    clusters.iloc[i] = i
clusters.astype(int).to_dict()

data = pd.Series(kmodel.predict(data.values.reshape(-1, 1)), name='肝气郁结证型系数').map(clusters.astype(int).to_dict())
data.head()

0    0
1    3
2    0
3    2
4    1
Name: 肝气郁结证型系数, dtype: int64

### 属性构造

利用已有的属性集合构造出新的属性，加入到现有的属性集中，提高数据挖掘的精度。

### 小波转换

一种非平稳信号的时频分析手段，通过伸缩和平移等运算过程对信号进行多尺度的聚焦分析，可以由粗及细地逐步观察信号，从中提取有用的信息。

## 数据规约

产生更小且保持原数据完整性的新数据集，在规约后的数据集上进行分析和挖掘将更有效率。

- 降低无效、错误数据对建模的影响，提高建模的准确性
- 大幅度缩减建模所用时间
- 降低储存数据的成本

### 属性规约

- 合并属性，将一些属性合并为新属性，抛弃旧属性
- 逐步向前选择
- 逐步向后删除
- 简单决策树归纳：一个简单决策树，删除掉没有被使用的属性
- 随机森林筛选
- 主成分分析

逐步向前选择、逐步向后删除、决策树归纳等都是直接删除不相关属性。

主成分分析是构造一个原始数据的正交变换，新空间的基底去掉了原数据中的相关性。**在应用中，通常是选出比原始变量个数少，能解释大部分数据中的变量的新变量，即所谓主成分，来代替原始变量进行建模**。

In [11]:
from sklearn.decomposition import PCA

In [26]:
inputfile = './data/data_4/principal_component.xls'

data = pd.read_excel(inputfile, header=None)

pca = PCA().fit(data)
print('explained ratios: {}'.format(np.cumsum(pca.explained_variance_ratio_)))
print('number of components: {}'.format(len(pca.components_)))

explained ratios: [0.77401126 0.93096071 0.97372013 0.99778605 0.99928883 0.99969982
 0.99990754 1.        ]
number of components: 8


In [27]:
pca = PCA(n_components=3).fit(data)
low_d = pca.transform(data)
low_d

array([[  8.19133694,  16.90402785,   3.90991029],
       [  0.28527403,  -6.48074989,  -4.62870368],
       [-23.70739074,  -2.85245701,  -0.4965231 ],
       [-14.43202637,   2.29917325,  -1.50272151],
       [  5.4304568 ,  10.00704077,   9.52086923],
       [ 24.15955898,  -9.36428589,   0.72657857],
       [ -3.66134607,  -7.60198615,  -2.36439873],
       [ 13.96761214,  13.89123979,  -6.44917778],
       [ 40.88093588, -13.25685287,   4.16539368],
       [ -1.74887665,  -4.23112299,  -0.58980995],
       [-21.94321959,  -2.36645883,   1.33203832],
       [-36.70868069,  -6.00536554,   3.97183515],
       [  3.28750663,   4.86380886,   1.00424688],
       [  5.99885871,   4.19398863,  -8.59953736]])